In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
import numpy as np
import nltk
from pyprojroot import here

from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit

path_data = here("./data")
os.chdir(path_data)
data_ra = pd.read_csv("ra_data.csv")
data_ra = clean_columns(data_ra)

nltk.download("vader_lexicon")
data_ra["date"] = pd.to_datetime(data_ra["date"])

11 column names have been cleaned

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/ipykernel_27975/814015519.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_ra["date"] = pd.to_datetime(data_ra["date"])


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("wordnet")


lemmatizer = WordNetLemmatizer()


# Function to preprocess text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    filtered_text = [
        lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words
    ]
    return " ".join(filtered_text)


# Preprocess the reviews
data_ra["reviews_clean"] = (
    data_ra["reviews"].apply(preprocess_text).values
)

data_ra.to_csv("data_ra_sentiment.csv")

In [3]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

# Let's assume your DataFrame is called df and the column with the text data is 'reviews'
data_ra["sentiment_scores"] = data_ra["reviews"].apply(
    lambda review: sia.polarity_scores(review)
)

# The above will return a dictionary with 'compound', 'neg', 'neu', 'pos' as keys.
# If you're interested in overall sentiment, you could use 'compound' score which is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive).
# You can create a new column for this score.

data_ra["compound_score"] = data_ra["sentiment_scores"].apply(
    lambda score_dict: score_dict["compound"]
)

In [42]:
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification")

# Define the candidate labels
candidate_labels = ["pain management"]


# Function to apply the classifier to a review
def classify_review(review):
    result = classifier(review, candidate_labels)
    return dict(zip(result["labels"], result["scores"]))


# Apply the classifier to each review
data_ra["emotion_scores"] = data_ra["reviews"].apply(
    classify_review
)

# Create a new column for each label
for label in candidate_labels:
    data_ra[label] = data_ra["emotion_scores"].apply(
        lambda scores: scores.get(label)
    )

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [50]:
data_ra_sentiment = clean_columns(data_ra)
len_pain = (
    data_ra_sentiment.query("pain_management >= .75").reset_index(drop=True).shape[0]
)
len_all = data_ra_sentiment.shape[0]
percent_pain = round(len_pain / len_all, 2)
data_pain = pd.DataFrame(
    {"len_pain": len_pain, "len_all": len_all, "percent_pain": percent_pain}, index=[0]
)

data_pain

0 column names have been cleaned

,len_pain,len_all,percent_pain
0,92,185,0.5


In [57]:
data_ra_pain = data_ra_sentiment.query("pain_management >= .75").reset_index(drop=True)

In [65]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bertopic import BERTopic

# Download the necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))

# Add custom words to stop words
custom_stop_words = ["humira", "syringe", "kit", "pen", "injector"]
stop_words.update(custom_stop_words)

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()


# Function to preprocess text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Tokenize the text
    word_tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    filtered_text = [
        lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words
    ]
    return " ".join(filtered_text)


# Preprocess the reviews
reviews = data_ra_pain["reviews_clean"].apply(preprocess_text).values

# Create BERTopic model
topic_model = BERTopic(language="english")

# Fit the model to the reviews
topics, _ = topic_model.fit_transform(reviews)

# Get the topic frequencies
topic_info = topic_model.get_topic_info()

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [70]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,29,-1_pain_ve_injection_month,"[pain, ve, injection, month, nt, year, taking,...",[since january 2009 taking without mtx taking ...
1,0,49,0_pain_year_injection_ra,"[pain, year, injection, ra, life, nt, methotre...",[diagnosed 4 year ago started methotrexate emb...
2,1,14,1_sting_great_pain_year,"[sting, great, pain, year, using, painful, fla...",['ve using 3 year . made remarkable improvemen...
